In [3]:
# augment_transpose.py

import mido
import os
import glob
from tqdm import tqdm

# --- Configuration ---
SOURCE_DIRECTORY = r"C:\Users\Hyperbook\Desktop\STUDIA\SEM III\PZ#2\dataset\lofi-dataset"
# We'll create a new directory for the augmented files to keep things clean.
OUTPUT_DIRECTORY = os.path.join(os.path.dirname(SOURCE_DIRECTORY), "lofi-dataset-augmented")
TRANSPOSE_SEMITONES = -2 # Transpose up by 1 semitone. Change to 2 for a whole tone.
FILENAME_SUFFIX = "_-2_semitone_up"

def transpose_dataset(source_dir, output_dir, semitones, suffix):
    """
    Augments a MIDI dataset by transposing each file up by a given number of semitones.
    """
    if not os.path.exists(source_dir):
        print(f"Error: Source directory not found at '{source_dir}'")
        return

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    print(f"Augmented files will be saved to: {output_dir}")

    midi_files = glob.glob(os.path.join(source_dir, '*.mid')) + \
                 glob.glob(os.path.join(source_dir, '*.midi'))

    if not midi_files:
        print(f"No MIDI files found in '{source_dir}'")
        return

    print(f"Found {len(midi_files)} files to augment.")

    for file_path in tqdm(midi_files, desc="Transposing MIDI files"):
        try:
            mid = mido.MidiFile(file_path, clip=True)
            
            # Create a new MIDI file to store the transposed version
            new_mid = mido.MidiFile(ticks_per_beat=mid.ticks_per_beat, type=mid.type)

            for track in mid.tracks:
                new_track = mido.MidiTrack()
                for msg in track:
                    new_msg = msg.copy()
                    
                    # Check if the message is a note event
                    if msg.type == 'note_on' or msg.type == 'note_off':
                        # Transpose the note, ensuring it stays within the valid MIDI range [0, 127]
                        new_note_value = msg.note + semitones
                        if 0 <= new_note_value <= 127:
                            new_msg.note = new_note_value
                        else:
                            # If transposition goes out of range, we keep the original note
                            # to avoid errors. Another option could be to skip the note.
                            pass
                    
                    new_track.append(new_msg)
                
                new_mid.tracks.append(new_track)

            # --- Create new filename and save the transposed file ---
            base_filename, extension = os.path.splitext(os.path.basename(file_path))
            new_filename = f"{base_filename}{suffix}{extension}"
            output_path = os.path.join(output_dir, new_filename)
            
            new_mid.save(output_path)

        except Exception as e:
            print(f"\nCould not process file {os.path.basename(file_path)}: {e}")

    print("\nDataset augmentation complete.")
    print(f"Original file count: {len(midi_files)}")
    print(f"New transposed files created: {len(midi_files)}")


if __name__ == '__main__':
    transpose_dataset(
        source_dir=SOURCE_DIRECTORY,
        output_dir=OUTPUT_DIRECTORY,
        semitones=TRANSPOSE_SEMITONES,
        suffix=FILENAME_SUFFIX
    )

Augmented files will be saved to: C:\Users\Hyperbook\Desktop\STUDIA\SEM III\PZ#2\dataset\lofi-dataset-augmented
Found 9164 files to augment.


Transposing MIDI files: 100%|██████████| 9164/9164 [01:13<00:00, 124.85it/s]


Dataset augmentation complete.
Original file count: 9164
New transposed files created: 9164
